In [1]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from pipeline.featurize import *
from pipeline.io import *
from run import *

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s - %(name)s.py]: %(message)s'
)

In [3]:
DaskClient()

distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-sfu74fl2', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-3_1chxf5', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-__4y8q_k', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-d5cs988b', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-pvwua0_t', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/isaak/repos/LAJC-expungement/dask-worker-space/worker-v62ys823', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 12.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39193,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 12.43 GiB
Comm: tcp://127.0.0.1:45325,Total threads: 2
Dashboard: http://127.0.0.1:45117/status,Memory: 4.14 GiB
Nanny: tcp://127.0.0.1:43231,


In [4]:
pd.set_option('max_columns', None)

In [5]:
config = ExpungeConfig.from_yaml('configs/default.yaml')

config.run_id

[INFO - pipeline.config.py]: Loading config from file: configs/default.yaml


'full_table_default'

In [6]:
query = (
    sa.select(expunge)
    .where(
        sa.and_(
            expunge.c.HearingDate < config.cutoff_date,
            # sa.or_(
            #     expunge.c.person_id == 1021000000606,
            #     expunge.c.person_id == 1070000000994,
            #     expunge.c.person_id == 163010000000348
                # expunge.c.person_id == 1001000000103
            # )
        )
    )
    .order_by(
        expunge.c.person_id, 
        expunge.c.HearingDate
    )
)

In [7]:
%%time
ddf = fetch_expunge_data(
    config,
    custom_query=query,
    # n_partitions=3
)

ddf.head()

[INFO - pipeline.io.py]: Reading from table: expunge_clean


CPU times: user 341 ms, sys: 139 ms, total: 480 ms
Wall time: 1.08 s


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [8]:
ddf

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
npartitions=1,,,,,,,,,
1.001000e+12,datetime64[ns],object,object,object,object,object,object,object,int64
3.662210e+14,...,...,...,...,...,...,...,...,...


In [9]:
ddf = clean_data(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [10]:
%%time
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

CPU times: user 483 ms, sys: 103 ms, total: 586 ms
Wall time: 2.13 s


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,date_if_conviction,date_if_felony_conviction,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_after_misdemeanor,next_conviction_disqualifier_after_felony,pending_after_misdemeanor,pending_after_felony,is_class_1_or_2,is_class_3_or_4,class1_2,class3_4
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False


In [11]:
ddf = classify_in_parallel(ddf, config).head()

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,date_if_conviction,date_if_felony_conviction,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_after_misdemeanor,next_conviction_disqualifier_after_felony,pending_after_misdemeanor,pending_after_felony,is_class_1_or_2,is_class_3_or_4,class1_2,class3_4,expungability,sameday_disqualifier,lifetime_disqualifier
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False,Petition,True,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False,Petition,True,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False,Petition,True,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False,Petition,True,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False,Petition,True,False


In [25]:
from pipeline.classify import *

In [12]:
ddf = remove_unneeded_columns(ddf)
ddf = append_run_id(ddf, config)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_after_misdemeanor,next_conviction_disqualifier_after_felony,pending_after_misdemeanor,pending_after_felony,class1_2,class3_4,run_id
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,full_table_default
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,full_table_default
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,full_table_default
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,full_table_default
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,full_table_default


In [13]:
%%time
file_paths = write_to_csv(ddf)

[INFO - classify.featurize.py]: Expungement feature data will be written to: /tmp/expunge_data
[INFO - classify.featurize.py]: Clearing any data from previous runs
[INFO - classify.featurize.py]: Command 'rm -rf /tmp/expunge_data/expunge_features-*.csv' returned with exit value: 0
[INFO - classify.featurize.py]: Executing Dask task graph and writing results to CSV...
[INFO - classify.featurize.py]: File(s) written successfully


CPU times: user 74.7 ms, sys: 0 ns, total: 74.7 ms
Wall time: 190 ms


In [14]:
file_paths[:5]

['/tmp/expunge_data/expunge_features-0.csv']

In [16]:
%%time
load_to_db(file_paths, config)

[INFO - classify.featurize.py]: Opening connection to PostGres via Psycopg
[INFO - classify.featurize.py]: Deleting any records with run_id: full_table_default
[INFO - classify.featurize.py]: Loading from file: /tmp/expunge_data/expunge_features-0.csv
[INFO - classify.featurize.py]: Load to DB complete


CPU times: user 12 ms, sys: 1.87 ms, total: 13.9 ms
Wall time: 19.2 ms
